In [1]:
import pyspark
from pyspark import SparkContext
from pyspark.sql.window import Window
from pyspark.sql import functions as F
from pyspark.sql import types as T

import json
import numpy as np
import time

In [2]:
def rename_columns(df, list_of_tuples):
    for (old_col, new_col) in list_of_tuples:
        df = df.withColumnRenamed(old_col, new_col)
    return df

def read_data(sqlContext, filepath):
    df = sqlContext.read.format("com.databricks.spark.csv")\
        .option("header", "true")\
        .option("inferSchema", "true") \
        .option("nullValue", "-")\
        .load(filepath)

    return df

def read_buste_data_v3(filepath, sqlContext):
    data_frame = sqlContext.read.csv(filepath, header=False,
                                     inferSchema=True,nullValue="-")
    while len(data_frame.columns) < 16:
        col_name = "_c" + str(len(data_frame.columns))
        data_frame = data_frame.withColumn(col_name, F.lit(None))

    data_frame = rename_columns(
        data_frame,
        [
            ("_c0", "route"),
            ("_c1", "tripNum"),
            ("_c2", "shapeId"),
            ("_c3", "shapeSequence"),
            ("_c4", "shapeLat"),
            ("_c5", "shapeLon"),
            ("_c6", "distanceTraveledShape"),
            ("_c7", "busCode"),
            ("_c8", "gpsPointId"),
            ("_c9", "gpsLat"),
            ("_c10", "gpsLon"),
            ("_c11", "distanceToShapePoint"),
            ("_c12", "timestamp"),
            ("_c13", "busStopId"),
            ("_c14", "problem"),
            ("_c15", "userBirthdate"),
            ("_c16", "cardTimestamp"),
            ("_c17", "lineName"),
            ("_c18", "cardNum"),
            ("_c19", "userGender"),
        ]
    )

    date = "-".join(filepath.split("/")[-2].split("_")[:3])

    data_frame = data_frame.withColumn("date", F.lit(date))
    data_frame = data_frame.withColumn("date", F.unix_timestamp(F.date_sub(F.col("date"),1),'yyyy-MM-dd'))
    
    return data_frame

def read_buste_data_old(filepath, sqlContext):
    data_frame = sqlContext.read.csv(filepath, header=False,
                                     inferSchema=True,nullValue="-")
    while len(data_frame.columns) < 16:
        col_name = "_c" + str(len(data_frame.columns))
        data_frame = data_frame.withColumn(col_name, F.lit(None))

    data_frame = rename_columns(
        data_frame,
        [
            ("_c0", "route"),
            ("_c1", "tripNum"),
            ("_c2", "shapeId"),
            ("_c3", "shapeSequence"),
            ("_c4", "shapeLat"),
            ("_c5", "shapeLon"),
            ("_c6", "distanceTraveledShape"),
            ("_c7", "busCode"),
            ("_c8", "gpsPointId"),
            ("_c9", "gpsLat"),
            ("_c10", "gpsLon"),
            ("_c11", "distanceToShapePoint"),
            ("_c12", "timestamp"),
            ("_c13", "busStopId"),
            ("_c14", "problem"),
            ("_c15", "numPassengers")
        ]
    )

    date = "-".join(filepath.split("/")[-2].split("_")[:3])

    data_frame = data_frame.withColumn("date", F.lit(date))
    data_frame = data_frame.withColumn("date", F.from_unixtime(
        F.unix_timestamp(F.date_sub(F.col("date"),1),'yyyy-MM-dd'),'yyyy-MM-dd'))
    
    return data_frame

def print_df(df,l=10):
    return df.limit(l).toPandas()

### Read Pre-OD Data

In [3]:
sc = SparkContext.getOrCreate()
sqlContext = pyspark.SQLContext(sc)

In [4]:
exp_data_folder_path = '/local/tarciso/masters/experiments/preliminary-exp/preliminary-exp-sample-data/'
pre_od_data = read_data(sqlContext,exp_data_folder_path + 'pre_od_matrix') \
                .withColumn('o_date',F.from_unixtime(F.unix_timestamp(F.col('o_date'),'yyyy-MM-dd'), 'yyyy-MM-dd')) \
                .withColumn('next_o_date',F.from_unixtime(F.unix_timestamp(F.col('next_o_date'),'yyyy-MM-dd'), 'yyyy-MM-dd')) \
                .orderBy(['cardNum','o_date','o_timestamp'])

In [5]:
print_df(pre_od_data)

,o_route,o_bus_code,o_date,o_tripNum,o_timestamp,o_shape_id,o_shape_seq,o_shape_lat,o_shape_lon,o_stop_id,...,next_o_timestamp,next_o_shape_id,next_o_shape_seq,next_o_shape_lat,next_o_shape_lon,next_o_stop_id,next_o_boarding_id,o_unixtimestamp,next_o_unixtimestamp,leg_duration
0,175,BC289,2017-05-10,10,17:02:45,1743,5444509,-25.451133,-49.252930,31749,...,17:46:02,2789,5510463,-25.432528,-49.272411,26355,1,72165,74762,43.283333
1,370,BC032,2017-05-10,9,17:46:02,2789,5510463,-25.432528,-49.272411,26355,...,17:02:45,1743,5444509,-25.451133,-49.252930,31749,0,74762,72165,-1.000000
2,372,BC312,2017-05-10,2,06:55:21,1891,6136884,-25.427901,-49.263238,29914,...,17:09:27,1719,5305706,-25.452114,-49.223827,14404,3,35721,72567,614.100000
3,50,LB602,2017-05-10,6,17:09:27,1719,5305706,-25.452114,-49.223827,14404,...,06:55:21,1891,6136884,-25.427901,-49.263238,29914,2,72567,35721,-1.000000
4,542,GA124,2017-05-10,6,07:01:40,1988,6403964,-25.550431,-49.265282,31256,...,17:05:57,1785,5950930,-25.466362,-49.279723,29165,5,36100,72357,604.283333
5,216,CA600,2017-05-10,7,17:05:57,1785,5950930,-25.466362,-49.279723,29165,...,07:01:40,1988,6403964,-25.550431,-49.265282,31256,4,72357,36100,-1.000000
6,533,EA166,2017-05-10,3,07:50:18,2872,6247890,-25.516804,-49.230914,30037,...,16:47:25,3260,6011921,-25.447553,-49.225671,32704,7,39018,71245,537.116667
7,20,KB605,2017-05-10,6,16:47:25,3260,6011921,-25.447553,-49.225671,32704,...,07:50:18,2872,6247890,-25.516804,-49.230914,30037,6,71245,39018,-1.000000
8,779,JC865,2017-05-10,1,05:55:17,2904,5602222,-25.459817,-49.319788,33567,...,15:02:51,2194,4299859,-25.435150,-49.273297,26149,9,32117,64971,547.566667
9,777,JC008,2017-05-10,8,15:02:51,2194,4299859,-25.435150,-49.273297,26149,...,05:55:17,2904,5602222,-25.459817,-49.319788,33567,8,64971,32117,-1.000000


In [6]:
#pre_od_data.columns

### Reading Bus Trip Data

In [43]:
old_exp_data_folder_path = '/local/tarciso/masters/experiments/preliminary-exp/preliminary-exp-sample-data/back-bulma-output'
bus_trip_data = read_buste_data_old(old_exp_data_folder_path + '/2017_05_11_veiculos.csv/part-00000', sqlContext)

In [44]:
bus_trip_data = bus_trip_data.withColumn("gps_timestamp", F.concat(F.col("date"), F.lit(" "), F.col("timestamp")))
bus_trip_data = bus_trip_data.withColumn("gps_timestamp_in_secs", F.unix_timestamp(F.col("gps_timestamp"), "yyyy-MM-dd HH:mm:ss"))

In [45]:
bus_trip_data = bus_trip_data.na.drop(subset=["route","busCode","busStopId","gps_timestamp_in_secs","tripNum"]) \
                            .dropDuplicates(['route','busCode','tripNum','busStopId']) \
                            .orderBy(['route','busCode','tripNum','gps_timestamp_in_secs']) \
                            .withColumn('id',F.monotonically_increasing_id()) \
                            .withColumn('route', F.col('route').cast(T.IntegerType()))

In [10]:
#bus_trip_data = bus_trip_data.na.drop(subset=["route","busCode","busStopId","gps_timestamp_in_secs","tripNum"]) \
#                             .withColumn('route', F.col('route').cast(T.IntegerType()))

In [11]:
#print_df(bus_trip_data)

### Trying a different approach

In [13]:
pre_od_data.printSchema()

root
 |-- o_route: integer (nullable = true)
 |-- o_bus_code: string (nullable = true)
 |-- o_date: string (nullable = true)
 |-- o_tripNum: integer (nullable = true)
 |-- o_timestamp: string (nullable = true)
 |-- o_shape_id: integer (nullable = true)
 |-- o_shape_seq: integer (nullable = true)
 |-- o_shape_lat: double (nullable = true)
 |-- o_shape_lon: double (nullable = true)
 |-- o_stop_id: integer (nullable = true)
 |-- o_boarding_id: long (nullable = true)
 |-- cardNum: integer (nullable = true)
 |-- next_o_route: integer (nullable = true)
 |-- next_o_bus_code: string (nullable = true)
 |-- next_o_date: string (nullable = true)
 |-- next_o_tripNum: integer (nullable = true)
 |-- next_o_timestamp: string (nullable = true)
 |-- next_o_shape_id: integer (nullable = true)
 |-- next_o_shape_seq: integer (nullable = true)
 |-- next_o_shape_lat: double (nullable = true)
 |-- next_o_shape_lon: double (nullable = true)
 |-- next_o_stop_id: integer (nullable = true)
 |-- next_o_boarding_i

In [14]:
bus_trip_data.printSchema()

root
 |-- route: integer (nullable = true)
 |-- tripNum: integer (nullable = true)
 |-- shapeId: integer (nullable = true)
 |-- shapeSequence: integer (nullable = true)
 |-- shapeLat: double (nullable = true)
 |-- shapeLon: double (nullable = true)
 |-- distanceTraveledShape: double (nullable = true)
 |-- busCode: string (nullable = true)
 |-- gpsPointId: string (nullable = true)
 |-- gpsLat: double (nullable = true)
 |-- gpsLon: double (nullable = true)
 |-- distanceToShapePoint: double (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- busStopId: integer (nullable = true)
 |-- problem: string (nullable = true)
 |-- numPassengers: integer (nullable = true)
 |-- date: string (nullable = true)
 |-- gps_timestamp: string (nullable = true)
 |-- gps_timestamp_in_secs: long (nullable = true)



In [141]:
#sample_size = 10
#pre_od_data_sample = pre_od_data.limit(sample_size)
pre_od_data_sample = pre_od_data

In [142]:
cond = [bus_trip_data.route == pre_od_data_sample.o_route, 
        bus_trip_data.busCode == pre_od_data_sample.o_bus_code, 
        bus_trip_data.date == pre_od_data_sample.o_date,
        bus_trip_data.tripNum == pre_od_data_sample.o_tripNum]
#od_trips = bus_trip_data.join(pre_od_data_sample, (((bus_trip_data.route == pre_od_data_sample.o_route) & (bus_trip_data.busCode == pre_od_data_sample.o_bus_code)) & ((bus_trip_data.date == pre_od_data_sample.o_date) & (bus_trip_data.tripNum == pre_od_data_sample.o_tripNum))), 'inner')
od_trips = bus_trip_data.join(pre_od_data_sample, cond, 'left_outer')

In [143]:
print_df(od_trips.orderBy(['route','busCode','tripNum','gps_timestamp_in_secs']))

,route,tripNum,shapeId,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,gpsLat,...,next_o_timestamp,next_o_shape_id,next_o_shape_seq,next_o_shape_lat,next_o_shape_lon,next_o_stop_id,next_o_boarding_id,o_unixtimestamp,next_o_unixtimestamp,leg_duration
0,NaN,1,2934,6446501,-25.452183,-49.224040,0.000,DC852,None,-25.452220,...,None,None,None,None,None,None,None,None,None,None
1,NaN,1,2934,6447496,-25.450932,-49.219970,432.312,DC852,None,-25.450976,...,None,None,None,None,None,None,None,None,None,None
2,NaN,1,2934,6447510,-25.448710,-49.220020,739.771,DC852,None,-25.448616,...,None,None,None,None,None,None,None,None,None,None
3,NaN,1,2934,6447525,-25.446724,-49.221252,1001.839,DC852,None,-25.446655,...,None,None,None,None,None,None,None,None,None,None
4,NaN,1,2934,6447538,-25.444639,-49.223163,1393.901,DC852,None,-25.444756,...,None,None,None,None,None,None,None,None,None,None
5,NaN,1,2934,6447574,-25.440200,-49.221386,2247.402,DC852,None,-25.440206,...,None,None,None,None,None,None,None,None,None,None
6,1.0,1,2938,6451385,-25.428133,-49.264531,338.081,BN997,None,-25.428198,...,None,None,None,None,None,None,None,None,None,None
7,1.0,1,2938,6451394,-25.430622,-49.263426,635.815,BN997,None,-25.430830,...,None,None,None,None,None,None,None,None,None,None
8,1.0,1,2938,6451403,-25.433455,-49.262219,972.378,BN997,None,NaN,...,None,None,None,None,None,None,None,None,None,None
9,1.0,1,2938,6451415,-25.435452,-49.265462,1428.729,BN997,None,-25.435406,...,None,None,None,None,None,None,None,None,None,None


In [144]:
print od_trips.count()

2811557


In [145]:
#trips_with_boardings = od_trips.filter(((od_trips.route == 20) & (od_trips.tripNum == 6)) & (od_trips.busCode == 'KB605')) \
#                                .select(['cardNum','date','route','busCode','tripNum','busStopId','o_stop_id','shapeLat','shapeLon','timestamp','o_timestamp','next_o_stop_id','next_o_shape_lat','next_o_shape_lon'])
#print_df(trips_with_boardings,l=100)

In [146]:
trips_with_boardings = od_trips
print_df(trips_with_boardings,l=100)

,route,tripNum,shapeId,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,gpsLat,...,next_o_timestamp,next_o_shape_id,next_o_shape_seq,next_o_shape_lat,next_o_shape_lon,next_o_stop_id,next_o_boarding_id,o_unixtimestamp,next_o_unixtimestamp,leg_duration
0,180,4,2758,5996473,-25.470718,-49.289736,13136.571,HC302,None,-25.470676,...,11:20:32,1938.0,6361620.0,-25.435197,-49.271753,26174.0,5.153961e+10,52835.0,51632.0,-1.000000
1,180,4,2758,5996473,-25.470718,-49.289736,13136.571,HC302,None,-25.470676,...,18:25:43,1718.0,3369019.0,-25.400492,-49.330708,33119.0,5.583457e+11,54251.0,77143.0,381.533333
2,180,4,2758,5996473,-25.470718,-49.289736,13136.571,HC302,None,-25.470676,...,08:28:15,2054.0,6455079.0,-25.519667,-49.334359,33232.0,5.583457e+11,54254.0,41295.0,-1.000000
3,180,4,2758,5996473,-25.470718,-49.289736,13136.571,HC302,None,-25.470676,...,12:28:03,1747.0,5995730.0,-25.430121,-49.271918,26339.0,6.614250e+11,54119.0,55683.0,26.066667
4,180,4,2758,5996473,-25.470718,-49.289736,13136.571,HC302,None,-25.470676,...,12:04:02,2758.0,5996176.0,-25.398857,-49.271473,28758.0,8.074539e+11,54236.0,54242.0,0.100000
5,180,4,2758,5996473,-25.470718,-49.289736,13136.571,HC302,None,-25.470676,...,07:04:19,1753.0,6096204.0,-25.364452,-49.271717,29385.0,8.074539e+11,54242.0,36259.0,-1.000000
6,180,4,2758,5996473,-25.470718,-49.289736,13136.571,HC302,None,-25.470676,...,09:05:59,1753.0,6096415.0,-25.395231,-49.271285,29392.0,8.074539e+11,52902.0,43559.0,-1.000000
7,180,4,2758,5996473,-25.470718,-49.289736,13136.571,HC302,None,-25.470676,...,17:08:41,1884.0,6244013.0,-25.446973,-49.295122,30158.0,8.933532e+11,54598.0,72521.0,298.716667
8,180,4,2758,5996473,-25.470718,-49.289736,13136.571,HC302,None,-25.470676,...,06:49:51,2217.0,3958367.0,-25.429567,-49.357154,34002.0,9.105331e+11,54107.0,35391.0,-1.000000
9,180,4,2758,5996473,-25.470718,-49.289736,13136.571,HC302,None,-25.470676,...,10:58:40,2194.0,4299656.0,-25.458765,-49.316226,33571.0,9.792525e+11,51551.0,50320.0,-1.000000


In [147]:
def dist(lat_x, long_x, lat_y, long_y):
    return F.acos(
        F.sin(F.toRadians(lat_x)) * F.sin(F.toRadians(lat_y)) + 
        F.cos(F.toRadians(lat_x)) * F.cos(F.toRadians(lat_y)) * 
            F.cos(F.toRadians(long_x) - F.toRadians(long_y))
    ) * F.lit(6371.0)

In [148]:
trips_with_boardings = trips_with_boardings.withColumn('dist',dist(F.col('shapeLat'),F.col('shapeLon'),F.col('next_o_shape_lat'),F.col('next_o_shape_lon')))

In [149]:
print_df(trips_with_boardings,l=100)

,route,tripNum,shapeId,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,gpsLat,...,next_o_shape_id,next_o_shape_seq,next_o_shape_lat,next_o_shape_lon,next_o_stop_id,next_o_boarding_id,o_unixtimestamp,next_o_unixtimestamp,leg_duration,dist
0,2,11,3077,4843434,-25.424278,-49.274140,642.819,DN029,None,-25.424296,...,2758,5996320,-25.431580,-49.272737,26553,876173328416,59169,70678,191.816667,0.824018
1,2,11,3077,4843434,-25.424278,-49.274140,642.819,DN029,None,-25.424296,...,1718,3368687,-25.458186,-49.346834,34133,1709396983875,58375,40582,-1.000000,8.215547
2,2,11,3077,4843453,-25.427496,-49.274471,1054.248,DN029,None,-25.427505,...,2758,5996320,-25.431580,-49.272737,26553,876173328416,59169,70678,191.816667,0.486286
3,2,11,3077,4843453,-25.427496,-49.274471,1054.248,DN029,None,-25.427505,...,1718,3368687,-25.458186,-49.346834,34133,1709396983875,58375,40582,-1.000000,8.027451
4,2,11,3077,4843468,-25.429173,-49.273811,1254.778,DN029,None,-25.429265,...,2758,5996320,-25.431580,-49.272737,26553,876173328416,59169,70678,191.816667,0.288570
5,2,11,3077,4843468,-25.429173,-49.273811,1254.778,DN029,None,-25.429265,...,1718,3368687,-25.458186,-49.346834,34133,1709396983875,58375,40582,-1.000000,8.010550
6,2,11,3077,4843474,-25.430858,-49.273884,1502.935,DN029,None,-25.430956,...,2758,5996320,-25.431580,-49.272737,26553,876173328416,59169,70678,191.816667,0.140335
7,2,11,3077,4843474,-25.430858,-49.273884,1502.935,DN029,None,-25.430956,...,1718,3368687,-25.458186,-49.346834,34133,1709396983875,58375,40582,-1.000000,7.930116
8,2,11,3077,4843483,-25.431362,-49.275689,1693.965,DN029,None,NaN,...,2758,5996320,-25.431580,-49.272737,26553,876173328416,59169,70678,191.816667,0.297399
9,2,11,3077,4843483,-25.431362,-49.275689,1693.965,DN029,None,NaN,...,1718,3368687,-25.458186,-49.346834,34133,1709396983875,58375,40582,-1.000000,7.741260


In [153]:
w = Window().partitionBy(['cardNum','date','route','busCode','tripNum']).orderBy('dist')

od_matrix = trips_with_boardings.filter('timestamp > o_timestamp') \
                    .withColumn('rn', F.row_number().over(w)) \
                    .where(F.col('rn') == 1) \
                    .select(['route','tripNum','o_stop_id','o_timestamp','busStopId','timestamp','dist']) \
                    .orderBy(['route'])

In [154]:
print_df(od_matrix, l=100)

,route,tripNum,o_stop_id,o_timestamp,busStopId,timestamp,dist
0,1,14,31453,11:49:35,35216,11:55:11,7.456119
1,1,13,26166,11:31:38,35219,11:32:23,4.122188
2,1,6,31453,09:10:50,31454,09:11:34,0.474069
3,1,15,26375,06:58:19,26166,12:46:47,3.275503
4,1,19,31453,13:37:58,35216,13:41:31,0.278334
5,1,13,31453,12:04:48,29082,12:15:09,2.186727
6,1,5,35216,08:55:03,26375,09:02:01,5.460175
7,1,16,31454,13:06:23,26166,13:10:44,0.119740
8,1,7,31453,08:58:43,35219,09:10:59,0.021777
9,1,4,35219,07:52:52,29420,07:53:54,0.258713


In [128]:
print_df(pre_od_data_sample.limit(sample_size).orderBy('o_route'))

,o_route,o_bus_code,o_date,o_tripNum,o_timestamp,o_shape_id,o_shape_seq,o_shape_lat,o_shape_lon,o_stop_id,...,next_o_timestamp,next_o_shape_id,next_o_shape_seq,next_o_shape_lat,next_o_shape_lon,next_o_stop_id,next_o_boarding_id,o_unixtimestamp,next_o_unixtimestamp,leg_duration
0,20,KB605,2017-05-10,6,16:47:25,3260,6011921,-25.447553,-49.225671,32704,...,07:50:18,2872,6247890,-25.516804,-49.230914,30037,6,71245,39018,-1.000000
1,50,LB602,2017-05-10,6,17:09:27,1719,5305706,-25.452114,-49.223827,14404,...,06:55:21,1891,6136884,-25.427901,-49.263238,29914,2,72567,35721,-1.000000
2,175,BC289,2017-05-10,10,17:02:45,1743,5444509,-25.451133,-49.252930,31749,...,17:46:02,2789,5510463,-25.432528,-49.272411,26355,1,72165,74762,43.283333
3,216,CA600,2017-05-10,7,17:05:57,1785,5950930,-25.466362,-49.279723,29165,...,07:01:40,1988,6403964,-25.550431,-49.265282,31256,4,72357,36100,-1.000000
4,370,BC032,2017-05-10,9,17:46:02,2789,5510463,-25.432528,-49.272411,26355,...,17:02:45,1743,5444509,-25.451133,-49.252930,31749,0,74762,72165,-1.000000
5,372,BC312,2017-05-10,2,06:55:21,1891,6136884,-25.427901,-49.263238,29914,...,17:09:27,1719,5305706,-25.452114,-49.223827,14404,3,35721,72567,614.100000
6,533,EA166,2017-05-10,3,07:50:18,2872,6247890,-25.516804,-49.230914,30037,...,16:47:25,3260,6011921,-25.447553,-49.225671,32704,7,39018,71245,537.116667
7,542,GA124,2017-05-10,6,07:01:40,1988,6403964,-25.550431,-49.265282,31256,...,17:05:57,1785,5950930,-25.466362,-49.279723,29165,5,36100,72357,604.283333
8,777,JC008,2017-05-10,8,15:02:51,2194,4299859,-25.435150,-49.273297,26149,...,05:55:17,2904,5602222,-25.459817,-49.319788,33567,8,64971,32117,-1.000000
9,779,JC865,2017-05-10,1,05:55:17,2904,5602222,-25.459817,-49.319788,33567,...,15:02:51,2194,4299859,-25.435150,-49.273297,26149,9,32117,64971,547.566667


In [ ]:
trips_with_boardings.filter('timestamp > o_timestamp').groupBy(['cardNum','date','route','busCode','tripNum','o_stop_id','next_o_stop_id']).agg(F.min('dist')).show()

In [20]:
print_df(bus_trip_data.filter(((bus_trip_data.route == 175) & (bus_trip_data.busCode == 'BC289')) &
                             ((bus_trip_data.tripNum == 10) & (bus_trip_data.busStopId == 31749))))

,route,tripNum,shapeId,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,gpsLat,gpsLon,distanceToShapePoint,timestamp,busStopId,problem,numPassengers,date,gps_timestamp,gps_timestamp_in_secs
0,175,10,1743,5444509,-25.451133,-49.252930,8002.63,BC289,None,-25.451213,-49.252883,10.023218,17:01:16,31749,NO_PROBLEM,15,2017-05-10,2017-05-10 17:01:16,1494446476
1,175,10,2743,5404834,-25.451146,-49.252929,0.00,BC289,None,-25.450663,-49.252911,53.752724,17:05:54,31749,NO_PROBLEM,0,2017-05-10,2017-05-10 17:05:54,1494446754
